## Train Othello-GPT and save to `ckpts`

Use `jupyter nbconvert --execute --to notebook --allow-errors --ExecutePreprocessor.timeout=-1 train_gpt_othello.ipynb --inplace --output ckpts/checkpoint.ipynb` to run in background

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# make deterministic
# from mingpt.utils import set_seed
# set_seed(44)

In [3]:
from connect_four.connect_four import ConnectFour
from connect_four.connect_four_dataset import ConnectFourDataset, CharConnectFourDataset, DatasetPreprocessingConfig
import pickle
import time
from tqdm import tqdm
import torch
from mingpt.utils import sample
from mingpt.model import GPT, GPTConfig
from mingpt.trainer import Trainer, TrainerConfig

## Generate the dataset

In [4]:
# dataset = ConnectFourDataset(
#     data_size=110000,
#     train_size=110000,
#     rows_count=6,
#     columns_count=7
# )

# with open("connect_four/dataset/dataset_6x7_110000.pkl", "wb") as f:
#     pickle.dump(dataset.sequences, f)

## Load the existing dataset from a file

In [5]:
with open("connect_four/dataset/dataset_6x7_110000.pkl", "rb") as f:
    game_transcriptions = pickle.load(f)

## Create the data loader

In [6]:
cf_data = ConnectFourDataset(data_size=0, train_size=100000, games_to_use=game_transcriptions)
char_cf_dataset = CharConnectFourDataset(cf_data)
char_cf_dataset.chars, char_cf_dataset.config, char_cf_dataset.max_len

Dataset created has 100000 sequences, 8 unique words.


([-100, 0, 1, 2, 3, 4, 5, 6],
 DatasetPreprocessingConfig(to_model_repr={-100: 0, 0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7}, from_model_repr={0: -100, 1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6}, block_size=41, vocab_size=8),
 42)

## Initialize model, and optionally train it

In [13]:
device = torch.cuda.current_device()
mconf = GPTConfig(char_cf_dataset.config.vocab_size, char_cf_dataset.config.block_size, n_layer=2, n_head=8, n_embd=80)
model = GPT(mconf).to(device)

In [8]:
max_epochs = 2
# initialize a trainer instance and kick off training
t_start = time.strftime("_%Y%m%d_%H%M%S")
tconf = TrainerConfig(
    max_epochs=max_epochs, 
    batch_size=10,
    learning_rate=5e-4,
    lr_decay=True, 
    warmup_tokens=len(char_cf_dataset)*char_cf_dataset.config.block_size*5, 
    final_tokens=len(char_cf_dataset)*char_cf_dataset.config.block_size*max_epochs,
    num_workers=0, 
    ckpt_path=f"./ckpts/gpt_at{t_start}.ckpt", 
)
trainer = Trainer(model, char_cf_dataset, None, tconf)
device = trainer.device
trainer.train()

epoch 1 iter 9999: train loss 1.90940. lr 1.000000e-04: 100%|██████████| 10000/10000 [02:08<00:00, 78.09it/s]
epoch 2 iter 9999: train loss 1.91945. lr 2.000000e-04: 100%|██████████| 10000/10000 [02:09<00:00, 77.15it/s]


## Load trained model from `ckpts`

In [15]:
load_res = model.load_state_dict(torch.load("./ckpts/gpt_at_20230618_093325.ckpt"))
if torch.cuda.is_available():
    device = torch.cuda.current_device()
    model = model.to(device)

## Validate it: for what percentage of all partial games in validation set, the top-1 prediction is legal

In [9]:
def validate_model(
        dataset: ConnectFourDataset,
        dataset_config: DatasetPreprocessingConfig,
        model: GPT
):
    total_nodes = 0
    success_nodes = 0

    bar = tqdm(dataset.valid[:200])
    for whole_game in bar:
        length_of_whole_game = len(whole_game)
        for length_of_partial_game in range(1, length_of_whole_game):
            total_nodes += 1
            context = whole_game[:length_of_partial_game]
            x = torch.tensor([dataset_config.to_model_repr[s] for s in context], dtype=torch.long)[None, ...].to(device)
            y = sample(model, x, 1, temperature=1.0)[0]
            completion = [dataset_config.from_model_repr[int(i)] for i in y if i != -1]
            game_repr = ConnectFour()
            piece = 1
            for move in context:
                game_repr.apply_move(piece, move)
                piece = 2 if piece == 1 else 1
            if game_repr.is_move_possible(completion[-1]):
                success_nodes += 1
        bar.set_description(f"{success_nodes/total_nodes*100:.2f}% pass rate: {success_nodes}/{total_nodes} among all searched nodes")
    print(f"{success_nodes/total_nodes*100:.2f}% pass rate: {success_nodes}/{total_nodes} among all searched nodes")
    return success_nodes/total_nodes*100

In [16]:
validate_model(dataset=cf_data, dataset_config=char_cf_dataset.config, model=model)

100.00% pass rate: 4061/4061 among all searched nodes: 100%|██████████| 200/200 [00:18<00:00, 10.98it/s]

100.00% pass rate: 4061/4061 among all searched nodes


100.0